# Import

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from collections import Counter
import warnings
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import itertools
from copy import deepcopy
import collections
from sklearn.model_selection import train_test_split
import json
import random
import re
import torch.optim as optim
from collections import defaultdict
import kagglehub

In [14]:
from google.colab import files
uploaded = files.upload()

Saving all_data_niid_001_keep_0_train_9.json to all_data_niid_001_keep_0_train_9 (1).json


In [15]:
from google.colab import files
uploaded2 = files.upload()

Saving all_data_niid_001_keep_0_test_9.json to all_data_niid_001_keep_0_test_9 (1).json


In [19]:
print(list(uploaded2.keys()))

['all_data_niid_001_keep_0_test_9 (1).json']


In [18]:
import io
import json

# Assumendo che il file caricato sia 'filename.json'
data = json.load(io.BytesIO(uploaded['all_data_niid_001_keep_0_train_9 (1).json']))

In [22]:
test_data  = json.load(io.BytesIO(uploaded2['all_data_niid_001_keep_0_test_9 (1).json']))

In [24]:
import json
import torch
from torch.utils.data import DataLoader, TensorDataset

# Carica il file JSON
with open('all_data_niid_001_keep_0_train_9 (1).json', 'r') as file:
    data = json.load(file)



In [25]:
with open('all_data_niid_001_keep_0_test_9 (1).json', 'r') as f:  # Cambia il percorso con quello corretto
    test_data = json.load(f)

In [26]:
users = data['users']
num_samples = data['num_samples']
user_data = data['user_data']

## Vocab creation

In [27]:
print(user_data[users[0]]['x'])  # Stampa il contenuto per il primo utente


["ll ne'er be drunk whilst I live again, but in honest, civil, godly company, for ", 'y reasonable demands. I hope, sir, I will do as it shall become one that would d', "ft or a bolt on 't; 'slid, 'tis but venturing. No, she shall not dismay me. I ca", " pass'd; but women, indeed, cannot abide 'em; they are very ill-favour'd rough t", 'him, may: they may give the dozen white luces in their coat. I may quarter, coz.', "ll grow more contempt. But if you say 'marry her,' I will marry her; that I am f", "e in Latin then too; but 'tis no matter; I'll ne'er be drunk whilst I live again", "ill-favour'd rough things. Re-enter PAGE I'll eat nothing, I thank you, sir. Nay", "'ll not confess. Marry, sir, I have matter in my head against you; and against y", 'll do that that is reason. So I do, sir. Nay, I will do as my cousin Shallow say', "bolt on 't; 'slid, 'tis but venturing. No, she shall not dismay me. I care not f", ' heartily; I am very well. I am not a-hungry, I thank you, forsooth. Go, s

In [28]:
all_texts = ''.join([''.join(seq) for user in users for seq in user_data[user]['x']])
chars = sorted(set(all_texts))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}

# Aggiungi il carattere di padding
char_to_idx['<pad>'] = len(char_to_idx)
idx_to_char = {idx: ch for ch, idx in char_to_idx.items()}

In [29]:
print(chars)

[' ', '!', "'", ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'E', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [30]:
print(char_to_idx)

{' ': 0, '!': 1, "'": 2, ',': 3, '-': 4, '.': 5, ':': 6, ';': 7, '?': 8, 'A': 9, 'B': 10, 'C': 11, 'E': 12, 'G': 13, 'H': 14, 'I': 15, 'J': 16, 'L': 17, 'M': 18, 'N': 19, 'O': 20, 'P': 21, 'R': 22, 'S': 23, 'T': 24, 'U': 25, 'V': 26, 'W': 27, 'Y': 28, '[': 29, ']': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56, '<pad>': 57}


In [ ]:
print(idx_to_char)

## Covert data into indices

In [31]:
inputs = [[char_to_idx[char] for char in user_data[user]['x'][0]] for user in users]
targets = [[char_to_idx[char] for char in user_data[user]['y'][0]] for user in users]

## Creation of TensorDataset and DataLoader

In [33]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

# Supponendo che input_tensors e target_tensors siano liste di tensori
input_tensors = [torch.tensor(seq) for seq in inputs]
target_tensors = [torch.tensor([seq]) for seq in targets]  # Assicurati che ogni target sia incapsulato in un tensore

# Aggiungi il carattere di padding al vocabolario
chars = sorted(set(all_texts))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
char_to_idx['<pad>'] = len(char_to_idx)  # Indice unico per il padding
idx_to_char = {idx: ch for ch, idx in char_to_idx.items()}

# Applica il padding solo agli input
padded_inputs = pad_sequence(input_tensors, batch_first=True, padding_value=char_to_idx['<pad>'])

# Concatena i tensori target in un unico tensore
target_tensors = torch.cat(target_tensors, dim=0)

# Crea il TensorDataset
dataset = TensorDataset(padded_inputs, target_tensors)

# Crea il DataLoader
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [34]:
def tensor_to_string(tensor, idx_to_char):
    """Converte un tensore di indici in una stringa di caratteri."""
    return ''.join(idx_to_char[idx.item()] for idx in tensor)

# Model

In [35]:
import torch.nn.functional as F

class CharLSTM(nn.Module):
    """
    Character-level LSTM model for text processing tasks.
    Includes embedding, LSTM, and a fully connected output layer.
    We use:
    - embedding size equal to 8;
    - 2 LSTM layers, each with 256 nodes;
    - densely connected softmax output layer.

    We can avoid to use explicitly the softmax function in the model and
    keep a cross entropy loss function as a loss function.

    as mentioned in paper [2] (Sashank Reddi, Zachary Charles, Manzil Zaheer, Zachary Garrett, Keith Rush,
    Jakub Konečný, Sanjiv Kumar, H. Brendan McMahan; Adaptive Federated Optimization, 2021)
    """
    def __init__(self, vocab_size = 70, embedding_size = 8, lstm_hidden_dim = 256, seq_length=80):
        super(CharLSTM, self).__init__()
        self.seq_length = seq_length
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.lstm_hidden_dim = lstm_hidden_dim
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size)
        self.lstm1 = nn.LSTM(input_size=embedding_size, hidden_size=lstm_hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=lstm_hidden_dim, hidden_size=lstm_hidden_dim, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_dim, vocab_size)

    def forward(self, x, hidden):
        """
        Forward pass through the model.
        """
        # Layer 1: Embedding
        x = self.embedding(x)  # Output shape: (batch_size, seq_length, embedding_dim)

        # Layer 2: First LSTM
        x, _ = self.lstm1(x)  # Output shape: (batch_size, seq_length, lstm_hidden_dim)

        # Layer 3: Second LSTM
        x, hidden = self.lstm2(x)  # Output shape: (batch_size, seq_length, lstm_hidden_dim)

        # Layer 4: Fully Connected Layer
        x = self.fc(x)  # Output shape: (batch_size, seq_length, vocab_size)

        # Softmax Activation
        #x = self.softmax(x)  # Output shape: (batch_size, seq_length, vocab_size)
        return x[:, -1, :], hidden

    def init_hidden(self, batch_size):
        """Initializes hidden and cell states for the LSTM."""
        return (torch.zeros(2, batch_size, self.lstm_hidden_dim),
            torch.zeros(2, batch_size, self.lstm_hidden_dim))
        #2 is equal to the number of lstm layers!



# Training

In [36]:
# Funzione per convertire i caratteri in tensori di indici
def char_to_tensor(characters):
    indices = [char_to_idx[char] for char in characters]
    return torch.tensor(indices, dtype=torch.long)

# Prepara i dati di test
input_tensors = []
target_tensors = []
for user in data['users']:
    for entry in data['user_data'][user]['x']:
        input_tensors.append(char_to_tensor(entry[:-1]))  # Tutti i caratteri tranne l'ultimo
        target_tensors.append(char_to_tensor(entry[-1]))  # L'ultimo carattere come target

# Padding e creazione di DataLoader
padded_inputs = pad_sequence(input_tensors, batch_first=True, padding_value=char_to_idx[' '])  # Usa spazio per padding se non hai '<pad>'
targets = torch.cat(target_tensors)
dataset = TensorDataset(padded_inputs, targets)
for elem1, elem2 in dataset:
  elem2 = elem2.unsqueeze(0)

loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definizione del modello
model = CharLSTM(vocab_size=len(char_to_idx))
model.train()  # Imposta il modello in modalità training

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(DEVICE)  # Sposta il modello sul dispositivo corretto

# Definizione di loss e ottimizzatore
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Funzione di training
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        total_samples = 0

        for inputs, targets in dataloader:
            inputs = inputs.to(DEVICE)  # Sposta gli input sul dispositivo corretto
            targets = targets.to(DEVICE)  # Sposta i target sul dispositivo corretto

            # Resetti i gradienti esistenti
            optimizer.zero_grad()

            # Inizializza lo stato nascosto
            hidden = model.init_hidden(inputs.size(0))
            hidden = (hidden[0].to(DEVICE), hidden[1].to(DEVICE))  # Sposta lo stato nascosto sul dispositivo corretto

            # Forward pass
            outputs, _ = model(inputs, hidden)

            # Calcola la loss
            outputs_flat = outputs.view(-1, len(char_to_idx))
            targets_flat = targets.view(-1)
            loss = criterion(outputs_flat, targets_flat)

            # Backward pass e ottimizzazione
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_samples += targets_flat.size(0)

            # Calcolo dell'accuratezza
            _, predicted = outputs_flat.max(1)
            total_correct += (predicted == targets_flat).sum().item()

        # Calcola la loss media e l'accuratezza per l'epoca
        avg_loss = total_loss / len(dataloader)
        accuracy = total_correct / total_samples

        # Stampa la loss media e l'accuratezza per l'epoca
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

# Eseguire il training
train_model(model, loader, criterion, optimizer, num_epochs=20)


Epoch 1/60, Loss: 2.9721, Accuracy: 0.2253
Epoch 2/60, Loss: 2.5340, Accuracy: 0.3032
Epoch 3/60, Loss: 2.2687, Accuracy: 0.3637
Epoch 4/60, Loss: 1.9873, Accuracy: 0.4199
Epoch 5/60, Loss: 1.6435, Accuracy: 0.5051
Epoch 6/60, Loss: 1.2483, Accuracy: 0.6132
Epoch 7/60, Loss: 0.8954, Accuracy: 0.7257
Epoch 8/60, Loss: 0.6554, Accuracy: 0.8109
Epoch 9/60, Loss: 0.4717, Accuracy: 0.8730
Epoch 10/60, Loss: 0.4313, Accuracy: 0.8769
Epoch 11/60, Loss: 0.4061, Accuracy: 0.8806
Epoch 12/60, Loss: 0.3476, Accuracy: 0.9006
Epoch 13/60, Loss: 0.3779, Accuracy: 0.8833
Epoch 14/60, Loss: 0.3813, Accuracy: 0.8880
Epoch 15/60, Loss: 0.3574, Accuracy: 0.8938
Epoch 16/60, Loss: 0.3316, Accuracy: 0.8985
Epoch 17/60, Loss: 0.2828, Accuracy: 0.9135
Epoch 18/60, Loss: 0.3216, Accuracy: 0.9027
Epoch 19/60, Loss: 0.3304, Accuracy: 0.9003
Epoch 20/60, Loss: 0.3247, Accuracy: 0.8977
Epoch 21/60, Loss: 0.3564, Accuracy: 0.8869
Epoch 22/60, Loss: 0.3349, Accuracy: 0.8969
Epoch 23/60, Loss: 0.3240, Accuracy: 0.89

# Testing

In [40]:
# Funzione per convertire i caratteri in tensori di indici
def char_to_tensor(characters):
    indices = [char_to_idx[char] for char in characters]
    return torch.tensor(indices, dtype=torch.long)

# Prepara i dati di test
input_tensors = []
target_tensors = []
for user in test_data['users']:
    for entry in test_data['user_data'][user]['x']:
        input_tensors.append(char_to_tensor(entry[:-1]))  # Tutti i caratteri tranne l'ultimo
        target_tensors.append(char_to_tensor(entry[-1]))  # L'ultimo carattere come target

# Padding e creazione di DataLoader
padded_inputs = pad_sequence(input_tensors, batch_first=True, padding_value=char_to_idx[' '])  # Usa spazio per padding se non hai '<pad>'
targets = torch.cat(target_tensors)
test_dataset = TensorDataset(padded_inputs, targets)
for elem1, elem2 in test_dataset:
  elem2 = elem2.unsqueeze(0)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


# Testa il modello
correct = 0
total = 0
for inputs, targets in test_loader:
    inputs = inputs.to(DEVICE)  # Sposta gli input sul dispositivo corretto
    targets = targets.to(DEVICE)  # Sposta i target sul dispositivo corretto

    # Inizializza lo stato nascosto
    hidden = model.init_hidden(inputs.size(0))
    hidden = (hidden[0].to(DEVICE), hidden[1].to(DEVICE))  # Sposta lo stato nascosto sul dispositivo corretto

    outputs, _ = model(inputs, hidden)
    outputs_flat = outputs.view(-1, len(char_to_idx))
    targets_flat = targets.view(-1)
    _, predicted = outputs_flat.max(1)
    total += targets.size(0)
    correct += (predicted == targets_flat).sum().item()

print(f'Test Accuracy: {100 * correct / total}%')

Test Accuracy: 40.116279069767444%


In [42]:
for elem1, elem2 in test_dataset:
  # print(tensor_to_string(elem1, idx_to_char))
  # print(tensor_to_string(elem2.unsqueeze(0), idx_to_char))
  print(elem1)
  print(elem2)

tensor([31, 49,  0, 49, 45, 45, 44,  0, 47, 51, 31, 48, 48, 35, 42,  0, 31, 50,
         0, 39, 50,  0, 31, 49,  0, 31, 44, 55,  0, 43, 31, 44,  0, 39, 44,  0,
        12, 44, 37, 42, 31, 44, 34,  5,  0, 28, 45, 51,  0, 31, 48, 35,  0, 31,
        36, 48, 31, 39, 34,  3,  0, 39, 36,  0, 55, 45, 51,  0, 49, 35, 35,  0,
        50, 38, 35,  0, 32, 35, 31])
tensor(48)
tensor([51, 48,  2, 34,  0, 48, 45, 51, 37, 38,  0, 50, 38, 39, 44, 37, 49,  5,
         0, 22, 35,  4, 35, 44, 50, 35, 48,  0, 21,  9, 13, 12,  0, 15,  2, 42,
        42,  0, 35, 31, 50,  0, 44, 45, 50, 38, 39, 44, 37,  3,  0, 15,  0, 50,
        38, 31, 44, 41,  0, 55, 45, 51,  3,  0, 49, 39, 48,  5,  0, 19, 31, 55,
         3,  0, 46, 48, 31, 55,  0])
tensor(55)
tensor([ 0, 47, 51, 39, 50, 50, 31, 44, 33, 35,  3,  0, 45, 48,  0, 45, 32, 42,
        39, 37, 31, 50, 39, 45, 44,  4,  2,  9, 48, 43, 39, 37, 35, 48, 45,  5,
         2,  0,  9, 42, 42,  0, 38, 39, 49,  0, 49, 51, 33, 33, 35, 49, 49, 45,
        48, 49,  3,  0, 

In [50]:
for elem1, elem2 in loader:
  print(tensor_to_string(elem1[0], idx_to_char))
  print(tensor_to_string(elem2, idx_to_char))
  #print(elem1[0])
  #print(elem2)

ll ne'er be drunk whilst I live again, but in honest, civil, godly company, for
 
y reasonable demands. I hope, sir, I will do as it shall become one that would 
d
ft or a bolt on 't; 'slid, 'tis but venturing. No, she shall not dismay me. I c
a
 pass'd; but women, indeed, cannot abide 'em; they are very ill-favour'd rough 
t
him, may: they may give the dozen white luces in their coat. I may quarter, coz
.
ll grow more contempt. But if you say 'marry her,' I will marry her; that I am 
f
e in Latin then too; but 'tis no matter; I'll ne'er be drunk whilst I live agai
n
ill-favour'd rough things. Re-enter PAGE I'll eat nothing, I thank you, sir. Na
y
'll not confess. Marry, sir, I have matter in my head against you; and against 
y
ll do that that is reason. So I do, sir. Nay, I will do as my cousin Shallow sa
y
bolt on 't; 'slid, 'tis but venturing. No, she shall not dismay me. I care not 
f
 heartily; I am very well. I am not a-hungry, I thank you, forsooth. Go, sirrah
,
; but 'tis no ma

KeyboardInterrupt: 